In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
from keras_model import tune_model

In [51]:
best_model, best_hyperparameters = tune_model()

Trial 14 Complete [00h 01m 24s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 17m 08s

Search: Running Trial #15

Value             |Best Value So Far |Hyperparameter
96                |128               |units_lstm1
128               |64                |units_lstm2
1                 |3                 |num_linear_layers
0.0094784         |0.0089397         |learning_rate
64                |64                |units_linear_0
64                |64                |units_linear_1
128               |64                |units_linear_2
4                 |2                 |tuner/epochs
2                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
1                 |0                 |tuner/round
0010              |None              |tuner/trial_id

Epoch 3/4


/opt/anaconda3/envs/tensorflow_env/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


 203/1005 ━━━━━━━━━━━━━━━━━━━━ 29s 37ms/step - accuracy: 0.0736 - loss: 2.6278

In [33]:
import numpy as np
import pandas as pd
import keras_tuner as kt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import datetime

from sklearn.utils import resample
from sklearn.model_selection import train_test_split

from mfcc_label import prepare_data
from config import *
from joblib import dump, load

In [4]:
#from DNN_utils
def column_str_to_numpy(df, colname:str):
    # Given pd.DataFrame df, convert the column colname from string to numpy array.
    if isinstance(df.iloc[0][colname], str):
        df[colname]=df[colname].apply(lambda x: np.fromstring(x[1:-1], sep=' '))

In [5]:
# Read the data
df_train_val = pd.read_csv('processed_data/dnn_never_train.csv')
df_test = pd.read_csv('processed_data/dnn_never_test.csv')

In [6]:
# Some columns are recorded as string although they are arrays.
column_str_to_numpy(df_train_val, 'mfcc')
column_str_to_numpy(df_train_val, 'label')
column_str_to_numpy(df_test, 'mfcc')
column_str_to_numpy(df_test, 'label')

#Split the train set into train and validation sets.
# df_train_pre, df_val = train_test_split(df_train_val, test_size=0.2, random_state=42)
df_train_val.reset_index(drop=True, inplace=True)
# df_val.reset_index(drop=True, inplace=True)

# Create a single class label (type: int) which is the highest probability class in the label vector (type: 14x1 array).
df_train_val['single_class_label'] = df_train_val['label'].apply(lambda x: np.argmax(x))
# df_val['single_class_label'] = df_val['label'].apply(lambda x: np.argmax(x))
df_test['single_class_label'] = df_test['label'].apply(lambda x: np.argmax(x))

In [7]:
pd.set_option('display.max_colwidth', None)

def upsample_multiple_minority(df):

    # Assuming df is your dataframe
    # Identify majority and minority classes
    majority_classes = [13, 14]
    minority_classes = list(range(13))

    # Find the size of the smallest majority class
    majority_class_size = df['single_class_label'].value_counts().nlargest(2).iloc[1]
    
    # List to hold the upsampled dataframes
    list_df = []

    # Loop through each minority class and upsample
    for class_value in minority_classes:
        df_minority_class = df[df['single_class_label'] == class_value]
        df_minority_upsampled = resample(df_minority_class, 
                                        replace=True,     # sample with replacement
                                        n_samples=majority_class_size,    # to match majority class size
                                        random_state=123) # reproducible results
        list_df.append(df_minority_upsampled)

    # Append majority classes without change
    df_majority = df[df['single_class_label'].isin(majority_classes)]
    list_df.append(df_majority)

    # Concatenate all dataframes
    df_upsampled = pd.concat(list_df)
    return df_upsampled

df_train_val = upsample_multiple_minority(df_train_val)
df_train_val.reset_index(drop=True, inplace=True)
# display(df_train)

In [8]:
# Assuming df_train is already loaded with 'mfcc' and 'label' columns
X = np.array(df_train_val['mfcc'].tolist())
y = np.array(df_train_val['label'].tolist())

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
def reshape_lstm(X_train):
    """
    Reshape the input data for LSTM.
    """

    # If the shape is not (num_samples, timesteps, num_features), reshape it
    if len(X_train.shape) == 2:  # Assuming shape is (num_samples, num_features)
        X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

    return X_train

X_train = reshape_lstm(X_train)
X_val = reshape_lstm(X_val)

In [40]:
X.shape

(40163, 20)

In [41]:
X_train.shape[1]

1

In [42]:
X_train.shape[2]

20

In [39]:
def build_model(hp):
    model = Sequential()
    model.add(LSTM(
        units=hp.Int('units_lstm1', min_value=64, max_value=128, step=32),
        input_shape=(X_train.shape[1], X_train.shape[2]),
        return_sequences=True
    ))
    model.add(BatchNormalization())
    model.add(LSTM(
        units=hp.Int('units_lstm2', min_value=64, max_value=128, step=32)
    ))

    # Add optional linear layers
    for i in range(hp.Int('num_linear_layers', 0, 3)):
        model.add(Dense(
            units=hp.Int(f'units_linear_{i}', min_value=32, max_value=128, step=32),
            activation='relu'
        ))

    model.add(Dense(14, activation='softmax'))

    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    optimizer = Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [10]:
# Enable device placement logging
# tf.debugging.set_log_device_placement(True)

model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(64))
model.add(Dense(14, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


2024-05-20 18:27:28.297605: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2024-05-20 18:27:28.297629: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-20 18:27:28.297634: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-20 18:27:28.297651: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-20 18:27:28.297665: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/opt/anaconda3/envs/tensorflow_env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequ

In [35]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [36]:
# Verify the shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

X_train shape: (32130, 1, 20)
y_train shape: (32130, 14)
X_val shape: (8033, 1, 20)
y_val shape: (8033, 14)


In [38]:
# Tuner setup with batch size tuning
def model_fit_with_params(hp, model, X_train, y_train):
    return model.fit(
        X_train, y_train,
        epochs=10,
        validation_split=0.2,
        batch_size=hp.Int('batch_size', min_value=32, max_value=128, step=32)
    )

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='models',
    project_name='lstm_tuning',
    overwrite=True
)

# Perform the search with the customized model fitting function
tuner.search(
    X, y_train,
    epochs=10,
    validation_split=0.2,
    callbacks=[],
    fit= model_fit_with_params
)

# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Get the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hyperparameters.values}")


Trial 10 Complete [00h 00m 48s]
val_accuracy: 0.5370370149612427

Best val_accuracy So Far: 0.6055088639259338
Total elapsed time: 00h 07m 47s

Search: Running Trial #11

Value             |Best Value So Far |Hyperparameter
96                |128               |units_lstm1
128               |64                |units_lstm2
0.00010101        |0.00097802        |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
804/804 ━━━━━━━━━━━━━━━━━━━━ 25s 29ms/step - accuracy: 0.2330 - loss: 2.3912 - val_accuracy: 0.3870 - val_loss: 1.7893
Epoch 2/2
802/804 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3926 - loss: 1.7646

KeyboardInterrupt: 

In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hyperparameters.values}")


In [37]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[tensorboard_callback, early_stopping_callback],
    verbose=1
)

Epoch 1/50
503/503 ━━━━━━━━━━━━━━━━━━━━ 14s 27ms/step - accuracy: 0.3136 - loss: 2.0660 - val_accuracy: 0.4177 - val_loss: 1.5361
Epoch 2/50
503/503 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step - accuracy: 0.4568 - loss: 1.4665 - val_accuracy: 0.4803 - val_loss: 1.3948
Epoch 3/50
503/503 ━━━━━━━━━━━━━━━━━━━━ 14s 27ms/step - accuracy: 0.4884 - loss: 1.3727 - val_accuracy: 0.5192 - val_loss: 1.2977
Epoch 4/50
503/503 ━━━━━━━━━━━━━━━━━━━━ 14s 27ms/step - accuracy: 0.5332 - loss: 1.2727 - val_accuracy: 0.5420 - val_loss: 1.2328
Epoch 5/50
503/503 ━━━━━━━━━━━━━━━━━━━━ 14s 27ms/step - accuracy: 0.5519 - loss: 1.2102 - val_accuracy: 0.5680 - val_loss: 1.1964
Epoch 6/50
503/503 ━━━━━━━━━━━━━━━━━━━━ 15s 29ms/step - accuracy: 0.5741 - loss: 1.1706 - val_accuracy: 0.5789 - val_loss: 1.1543
Epoch 7/50
503/503 ━━━━━━━━━━━━━━━━━━━━ 14s 28ms/step - accuracy: 0.5963 - loss: 1.1160 - val_accuracy: 0.6115 - val_loss: 1.0936
Epoch 8/50
503/503 ━━━━━━━━━━━━━━━━━━━━ 14s 28ms/step - accuracy: 0.6031 - loss: 1.1082 - 

KeyboardInterrupt: 

In [21]:
# Save the model
model.save('models/lstm_never_trained.keras')

In [17]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

252/252 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7520 - loss: 0.8277
Validation Loss: 0.8389800190925598
Validation Accuracy: 0.7451761364936829


In [19]:
# evaluate the model on the test set
X_test = np.array(df_test['mfcc'].tolist())
y_test = np.array(df_test['label'].tolist())
X_test = reshape_lstm(X_test)

test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

540/540 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.6085 - loss: 1.3517
Test Loss: 1.354977011680603
Test Accuracy: 0.6057162880897522


In [20]:
def path_to_emission(file_path_phn: str, file_path_wav: str) -> np.ndarray:
    '''
    Given the path of a file, get the emission probabilities.
    Args:
        file_path: Path of the audio file as a string.
    Returns:
        emit: pd.dataframe
            Emission probabilities for each frame in the audio file.
    '''
    df_test = prepare_data(file_path_phn,file_path_wav)
    column_str_to_numpy(df_test, 'mfcc')
    column_str_to_numpy(df_test, 'label')

    X_test = np.array(df_test['mfcc'].tolist())
    # y_test = np.array(df_test['label'].tolist())
    X_test = reshape_lstm(X_test)
 
    emission_data = model.predict(X_test)

    return emission_data

In [23]:
test_result = path_to_emission(TEST_PHN_PATH, TEST_WAV_PATH)

For start 51600 and end 52000, there is no time-overlapping row.
Caution: There is no time-overlapping rows for start 51600 and end 52000
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [26]:
test_result.shape

(645, 14)

In [27]:
df = pd.DataFrame({'label': list(test_result)})
df.shape

(645, 1)

In [30]:
def get_emission_all_paths(path_type: str = 'test'):
    paths = load('processed_data/train_test_dataset_never.joblib')[path_type]
    data = {}
    for i in range(len(paths)):
        file_path_wav, file_path_phn, file_path_word = paths[i]
        emission_data = path_to_emission(file_path_phn, file_path_wav)
        emission_df = pd.DataFrame({'label': list(test_result)})
        data[(file_path_wav, file_path_phn, file_path_word)] = emission_df

    return data

data = get_emission_all_paths() 
dump(data, "processed_data/test_data_for_hmm_keras.joblib")

For start 36960 and end 37360, there is no time-overlapping row.
Caution: There is no time-overlapping rows for start 36960 and end 37360
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
For start 45760 and end 46160, there is no time-overlapping row.
Caution: There is no time-overlapping rows for start 45760 and end 46160
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For start 62960 and end 63360, there is no time-overlapping row.
Caution: There is no time-overlapping rows for start 62960 and end 63360
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
F

['processed_data/test_data_for_hmm_keras.joblib']